# Iterative Prisoner's Dilemma


## Description

The [Prisoner's Dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) (PD) is a classical game analyzed in game theory, which is widely used to (attempt to) model social/economical interaction. It's a "dilemma" as, if exploited to explain the emergence of altruism in human or in general animal society, it fails badly at a first glance.

The classical situation-representation of the PD is that of two prisoners whose conviction depends on their mutual cooperation. It is easier understood though if illustrated in terms of a trade-off game (closed bag exachange):

*Two people meet and exchange closed bags, with the understanding that one of them contains money, and the other contains a purchase. Either player can choose to honor the deal by putting into his or her bag what he or she agreed, or he or she can defect by handing over an empty bag.*

It is obvious that for both players the winning strategy is to NOT cooperate.

Things changes when the interaction between the two individuals is iterated, in that case a more altruist attitude (strategy) is expected to emerge. The goal of this project is to test this hypothesis.

Mathematically the PD can be expressed with very basic linear algebra. The key component is the **Payoff matrix** $M$, which quantify the reward each player gets depending on whether she cooperated or not (defect):

$$
M = 
\begin{pmatrix} 
R & S \\
T & P 
\end{pmatrix}
$$

with $T,R,S,P$ integers that satisfy the following conditions:

$$
T>R>P>S; \quad 2R > T+S
$$

for example $T=3$, $R=2$, $P=1$ and $S=0$, or  $T=5$, $R=3$, $P=2$, $S=0$. Each player choice (move) can be represented by one of the two axis in ${\rm I\!R}^2$, i.e. $u_C=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $u_D=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$, where the first coordinate stands for *Cooperate* and the second for *Defect*. Being $u_1$ and $u_2$ their rewards $r_1$ and $r_2$ can be computed then as:

$$
r_1 = u_1^T M u_2
\quad
\quad
r_2 = u_2^T M u_1
$$

In an Iterative Prisoner's Dilemma (IPD), two players play prisoner's dilemma more than once in succession and they remember previous actions of their opponent and change their strategy accordingly. The winning strategy is the one which yields to a larger reward at the end of the IPD.

The strategy can be represented as a function which outputs either $u_C$ or $u_D$. Such function can depend on the opponent's history of moves, her on history of moves, on the number of moves played till that moment and so on, but it can only be based on a probability density function. Possible strategies are:

* **Nice guy**: always cooperate (the function's output is always $u_D$)
* **Bad guy**: always defect 
* **Mainly nice**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k<50$
* **Mainly bad**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k>50$
* **tit-for-tat**: start by cooperating, then repeat what the opponent has done in the previous move 

Many more and much more complex strategies can be implemented. The strategy can even change during the IPD.


## Assignments

* Implement a simple IPD between two players implementing two given strategies. Study the evolution along the tournament confronting different strategies; study the overall outcome in the different configurations. 
* Implement a multiple players IPD (MPIPD) where several strategies play against each other in a roud-robin scheme
* Iterate what done in the previous task (repeated MPIPD, rMPIPD)  by increasing the population implementing a given strategy depending on the results that strategy achieved in the previous iteration
* Implement a rMPIPD where strategies are allowed to mutate. The goal is to simulate the effect of genetic mutations and the effect of natura selection. A parameter (gene) should encode the attidue of an individual to cooperate, such gene can mutate randomly and the corresponding phenotype should compete in the MPIPD such that the best-fitted is determined.

In [1]:
import numpy as np
import pandas as pd
import random

# Preliminary definition

In this section we will define all constants and function needed to the assignement

#### Constants

Let's define the constants needed for the assignement

In [2]:
# Definition of the matrix of the payoff
M = np.array([[2, 0], [3, 1]])

# Definition of the two possible moves
uc = np.array([1, 0])
ud = np.array([0, 1])

print('The matrix of the payoff:')
print(M)

# Definition of the probability for defining the two moves
probability_mainly_nice = 0.75
probability_mainly_bad = 0.75

# Definition of the number of stage in the repeated game
n = 10

The matrix of the payoff:
[[2 0]
 [3 1]]


#### Strategies

Disclamer, game theoretically speaking, this is not the formal way to define strategies. In fact in game theory all player decide in advance what move to play in every single circumstance, here we only care about what happenes on the moves' path.

In [3]:
def nice_guy(number_of_stage):
    strat = uc.copy()
    return np.broadcast_to(strat, (number_of_stage, 2))

def bad_guy(number_of_stage):
    strat = ud.copy()
    return np.broadcast_to(strat, (number_of_stage, 2))

def mainly_nice(number_of_stage):
    strat = uc.copy()
    strat = np.broadcast_to(strat, (number_of_stage, 2)).copy()
    for i in range(number_of_stage):
        if random.random() >= probability_mainly_nice:
            strat[i, :] = ud
    return strat

def mainly_bad(number_of_stage):
    strat = ud.copy()
    strat = np.broadcast_to(strat, (number_of_stage, 2)).copy()
    for i in range(number_of_stage):
        if random.random() >= probability_mainly_bad:
            strat[i, :] = uc
    return strat

def tit_for_tat(opponent_strategy, number_of_stage):
    if opponent_strategy is tit_for_tat:
        return nice_guy(number_of_stage), nice_guy(number_of_stage)
    o = opponent_strategy(number_of_stage)
    return np.vstack((uc, o[:-1])), o

def nash_equilibrium(number_of_stage):
    strat = uc.copy()
    strat = np.broadcast_to(strat, (number_of_stage, 2)).copy()
    strat[number_of_stage-1, :] = ud
    return strat


Game theoretically speaking, would be more effective to model mainly_bad and mainly_nice strategy with mixed strategy.

A mixed strategy expresses the fact that a player plays a strategy with a given probability, we can easily find the payoff of this strategy looking for the expected payoff, that is the linear combination of the payoff multiplied for the probability of playing that move.  

In [4]:
def mainly_nice_mixed(number_of_stage):
    move = probability_mainly_nice * uc + (1 - probability_mainly_nice) * ud
    return np.broadcast_to(move, (number_of_stage, 2))

def mainly_bad_mixed(number_of_stage):
    move = probability_mainly_bad * ud + (1 - probability_mainly_nice) * uc
    return np.broadcast_to(move, (number_of_stage, 2))


#### Play

Let's define a function that computes the payoff of a IPD

In [5]:
def play (first_strat, second_strat, number_of_stage, verbose=False):
    if first_strat is tit_for_tat:
        f_strat, s_strat = first_strat(second_strat, number_of_stage)
    elif second_strat is tit_for_tat:
        s_strat, f_strat = second_strat(first_strat, number_of_stage)
    else:
        f_strat = first_strat(number_of_stage)
        s_strat = second_strat(number_of_stage)
    out_first = []
    out_second = []
    if verbose:
        print('First player\'s strategy:\t\tSecond player\'s strategy:')
        for f_move, s_move in zip(f_strat, s_strat):
            print(f_move, '\t\t\t\t\t' +str(s_move))
        print()
    for f, s in zip(f_strat, s_strat):
        out_first += [f.dot(M).dot(s.T) , ]
        out_second += [s.dot(M).dot(f.T) , ]
    if verbose:
        print('The outcome of the first player is:')
        print(out_first, '\n')
        print('The outcome of the second player is:')
        print(out_second, '\n')
        print('The total outcome for the first player is:', np.array(out_first).sum())
        print('The total outcome for the second player is:', np.array(out_second).sum())
    return np.array(out_first).sum(), np.array(out_second).sum()
    

# First task

Implement a simple IPD between two players implementing two given strategies. Study the evolution along the tournament confronting different strategies; study the overall outcome in the different configurations.

In [6]:
print('Let\'s see what happenes if a mainly nice guy plays against a tit fot tat\n')
outcome = play(mainly_nice, tit_for_tat, n, verbose=True)

Let's see what happenes if a mainly nice guy plays against a tit fot tat

First player's strategy:		Second player's strategy:
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[0 1] 					[1 0]
[1 0] 					[0 1]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[0 1] 					[1 0]

The outcome of the first player is:
[2, 2, 2, 3, 0, 2, 2, 2, 2, 3] 

The outcome of the second player is:
[2, 2, 2, 0, 3, 2, 2, 2, 2, 0] 

The total outcome for the first player is: 20
The total outcome for the second player is: 17


As we can see in this specific total outcomes depend on the strategy adopted by the mainly nice guy, since it is not a deterministic strategy. But we can say that if the last move of the mainly nice guy is defect, the player will have a payoff grater than the opponet's one.

Let's run it a lot of time and let's see what happenes

In [7]:
repetition = 100000
mainly_nice_outcome = 0
tit_for_tat_outcome = 0
for i in range(repetition):
    outs = play(mainly_nice, tit_for_tat, n)
    mainly_nice_outcome += outs[0]
    tit_for_tat_outcome += outs[1]
mainly_nice_outcome = mainly_nice_outcome / repetition
tit_for_tat_outcome = tit_for_tat_outcome / repetition
print('Mainly Nice average outcome is:', mainly_nice_outcome)
print('Tit for tat average outcome is:', tit_for_tat_outcome)

Mainly Nice average outcome is: 17.9922
Tit for tat average outcome is: 17.24619


And let's compare this to the total outcome allowing mixed strategy

In [8]:
out = play(mainly_nice_mixed, tit_for_tat, n, verbose=True)

First player's strategy:		Second player's strategy:
[0.75 0.25] 					[1. 0.]
[0.75 0.25] 					[0.75 0.25]
[0.75 0.25] 					[0.75 0.25]
[0.75 0.25] 					[0.75 0.25]
[0.75 0.25] 					[0.75 0.25]
[0.75 0.25] 					[0.75 0.25]
[0.75 0.25] 					[0.75 0.25]
[0.75 0.25] 					[0.75 0.25]
[0.75 0.25] 					[0.75 0.25]
[0.75 0.25] 					[0.75 0.25]

The outcome of the first player is:
[2.25, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75] 

The outcome of the second player is:
[1.5, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75] 

The total outcome for the first player is: 18.0
The total outcome for the second player is: 17.25


As you can see on average they are pretty close, so from now on we consider only mixed strategy.

In [9]:
#TODO Prove that they are the same with the p-value

Let's keep on with some interesting comparisons:
* nice guy against bad guy
* bad guy against bad guy
* tit fot tat against bad guy
* nash equilibrium against nash equilibrium
* nash equilibrim against tit for tat

In [10]:
out = play(nice_guy, bad_guy, n, verbose=True)

First player's strategy:		Second player's strategy:
[1 0] 					[0 1]
[1 0] 					[0 1]
[1 0] 					[0 1]
[1 0] 					[0 1]
[1 0] 					[0 1]
[1 0] 					[0 1]
[1 0] 					[0 1]
[1 0] 					[0 1]
[1 0] 					[0 1]
[1 0] 					[0 1]

The outcome of the first player is:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

The outcome of the second player is:
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 

The total outcome for the first player is: 0
The total outcome for the second player is: 30


In [11]:
out = play(bad_guy, bad_guy, n, verbose=True)

First player's strategy:		Second player's strategy:
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]

The outcome of the first player is:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

The outcome of the second player is:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

The total outcome for the first player is: 10
The total outcome for the second player is: 10


In [12]:
out = play(tit_for_tat, bad_guy, n, verbose=True)

First player's strategy:		Second player's strategy:
[1 0] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]
[0 1] 					[0 1]

The outcome of the first player is:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

The outcome of the second player is:
[3, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

The total outcome for the first player is: 9
The total outcome for the second player is: 12


In [13]:
out = play(nash_equilibrium, nash_equilibrium, n, verbose=True)

First player's strategy:		Second player's strategy:
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[0 1] 					[0 1]

The outcome of the first player is:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 1] 

The outcome of the second player is:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 1] 

The total outcome for the first player is: 19
The total outcome for the second player is: 19


In [14]:
out = play(nash_equilibrium, tit_for_tat, n, verbose=True)

First player's strategy:		Second player's strategy:
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[1 0] 					[1 0]
[0 1] 					[1 0]

The outcome of the first player is:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 3] 

The outcome of the second player is:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 0] 

The total outcome for the first player is: 21
The total outcome for the second player is: 18


## Conclusions

From this we can conclude that the way to achieve the best payoff is to play bad guy against nice guy, but in this configuration the nice guy achieve the lowest possible payoff, so nobody will play nice guy. 

An improvement could be playing bad guy against bad guy, but in this case both payoffs are not so high, playing tit for tat against bad guy does not improve the payoff of the first player, but allow the second player to respond with tit for tat.

Finally, as we know from the theory we can say that the Nash Equilibrium is the best strategy to play for both player, it consists on collaborate for each round but the last one, in this case after the last round there is no punishment aviable, so the best choice for all the player is acting in a selfish way and defect. 

# Second task

Implement a multiple players IPD (MPIPD) where several strategies play against each other in a roud-robin scheme

#### Preliminary definition

Let's define the round robin scheme

In [15]:
# If you want to see really in details what is happening set verbose = True
def round_robin(strategies, labels, number_of_stage, verbose=False):
    payoffs = np.empty((len(strategies),len(strategies)), dtype=object)
    winners = np.empty((len(strategies),len(strategies)), dtype=str)
    for i, (f_strat, f_label) in enumerate(zip(strategies, labels)):
        for j, (s_strat, s_label) in enumerate(zip(strategies, labels)):
            if verbose:
                print('First player is', f_label, 'second player is', s_label)
            outcome = play(f_strat, s_strat, number_of_stage, verbose=verbose)
            if outcome[0] == outcome[1]:
                winners[i, j] = '\u25EF' # It is a circle
                if verbose:
                    print('\nPlayers draw\n\n')
            elif outcome[0] > outcome[1]:
                winners[i, j] = '\u2B60' # It is a left arrow
                if verbose:
                    print('\nPlayer', f_label, 'wins\n\n')
            else:
                winners[i,j] = '\u2B61' # It is a up arrow
                if verbose:
                    print('\nPlayer', s_label, 'wins\n\n')
            payoffs[i, j] = outcome
    payoffs = pd.DataFrame(payoffs, columns = strategy_labels , index= strategy_labels)
    winners = pd.DataFrame(winners, columns = strategy_labels , index= strategy_labels)
    return payoffs, winners
    

## Output

Let's see what round_robin produces, if you want to see the whole process set verbose = True.

Whatch out, inside the touple the first payoff is the payoff of the player on the left.

In [16]:
strategies = [nice_guy, bad_guy, mainly_nice_mixed, mainly_bad_mixed, tit_for_tat, nash_equilibrium]
strategy_labels =['Nice Guy','Bad Guy','Mainly Nice Mixed', 'Mainly Bad Mixed', 'Tit for Tat', 'Nash equilibrium']

payoffs, winners = round_robin(strategies, strategy_labels, n, verbose=False)

display(payoffs)
display(winners)

,Nice Guy,Bad Guy,Mainly Nice Mixed,Mainly Bad Mixed,Tit for Tat,Nash equilibrium
Nice Guy,"(20, 20)","(0, 30)","(15.0, 22.5)","(5.0, 27.5)","(20, 20)","(18, 21)"
Bad Guy,"(30, 0)","(10, 10)","(25.0, 2.5)","(15.0, 7.5)","(12, 9)","(28, 1)"
Mainly Nice Mixed,"(22.5, 15.0)","(2.5, 25.0)","(17.5, 17.5)","(7.5, 22.5)","(18.0, 17.25)","(20.5, 16.0)"
Mainly Bad Mixed,"(27.5, 5.0)","(7.5, 15.0)","(22.5, 7.5)","(12.5, 12.5)","(14.0, 11.75)","(25.5, 6.0)"
Tit for Tat,"(20, 20)","(9, 12)","(17.25, 18.0)","(11.75, 14.0)","(20, 20)","(18, 21)"
Nash equilibrium,"(21, 18)","(1, 28)","(16.0, 20.5)","(6.0, 25.5)","(21, 18)","(19, 19)"


,Nice Guy,Bad Guy,Mainly Nice Mixed,Mainly Bad Mixed,Tit for Tat,Nash equilibrium
Nice Guy,◯,⭡,⭡,⭡,◯,⭡
Bad Guy,⭠,◯,⭠,⭠,⭠,⭠
Mainly Nice Mixed,⭠,⭡,◯,⭡,⭠,⭠
Mainly Bad Mixed,⭠,⭡,⭠,◯,⭠,⭠
Tit for Tat,◯,⭡,⭡,⭡,◯,⭡
Nash equilibrium,⭠,⭡,⭡,⭡,⭠,◯
